In [ ]:
#importing required libraries
import pandas as pd
from urllib.parse import urlparse
import re

In [ ]:
#Loading the dataset
df_main = pd.read_csv("C:/Users/Insight/Desktop/Python_analysis/startup_funding.csv")
df = df_main.copy()
df.head()

In [ ]:
df.info() 

### Cleaning all columns

In [ ]:
#Renaming column names for consistency
def clean_column_names(df):
    df.columns = df.columns.str.replace('  ', ' ', regex=False)
    df.rename(columns = {'Sr No':'serial_no', 'Date dd/mm/yyyy':'disbursement_date', 'InvestmentnType':'investment_type'}, 
              inplace=True)
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    return df

# Apply the function to clean column names
df = clean_column_names(df)
df.head()

In [ ]:
#Checking columns with null values
df.isnull().sum()   

In [ ]:
df.isnull().mean()*100  

In [ ]:
df = df.drop(columns=['remarks', 'subvertical']) 

In [ ]:
#Selecting columns with null values
columns_to_fill = ['industry_vertical', 'city_location', 'investors_name', 'investment_type']

#Replacing the blank and NaN columns with "Unknown" 
df[columns_to_fill] = df[columns_to_fill].fillna('Unknown')
df.head()

In [ ]:
#List of unwanted characters/patterns to remove
unwanted_patterns = [r'\\xc2\\xa0', r'\\xc3\\x98', r'\\xe2\\x80\\x99', r'\\xc3\\xa9c', r'\\xe2\\x80\\x93', r'\\n', r'\\x',]

#Combining patterns into one big regex pattern (handles all unwanted characters at a go)
combined_pattern = re.compile('|'.join(unwanted_patterns))

#Function to clean unwanted patterns and also handle any encoding issues
def clean_text(text):
    if isinstance(text, str):
        text = bytes(text, 'utf-8').decode('unicode_escape') #This decodes escape sequences (such as \\xc2\\xa0), unwanted patterns into their respective characters (e.g., the non-breaking space character). 
        text = combined_pattern.sub('', text) #Removing unwanted patterns
        text = re.sub(r'\s+', ' ', text).strip() #Removing extra spaces that might have been created during cleaning
    return text

#Applying the cleaning function to all columns
df = df.applymap(clean_text)

specific_rows = [2605, 2609, 2611, 2227, 1186, 141]
df_selected = df.loc[specific_rows]
df_selected.head()

### Cleaning the date column

In [ ]:
#Function to clean and standardize dates

def clean_date(date):
    date = date.replace('.', '/').replace('//', '/') #Replacing incorrect separators
    
    #Fixing short years (e.g., '015' -> '2015')
    if date.count('/') == 2:
        day, month, year = date.split('/')
        if len(year) < 4:
            year = '20' + year[-2:]
        date = f"{day}/{month}/{year}"
    
    #Handling joined month and year (e.g., '05/072018' -> '05/07/2018')
    elif date.count('/') == 1 and len(date.split('/')[1]) == 6:
        day, month_year = date.split('/')
        month, year = month_year[:2], month_year[2:]
        date = f"{day}/{month}/{year}"

    #Converting to datetime and format as dd/mm/yyyy
    return pd.to_datetime(date, errors='coerce', dayfirst=True).strftime('%d/%m/%Y')

#Applying the function to clean and format the dates
df['disbursement_date'] = df['disbursement_date'].apply(clean_date)

specific_rows = [2571, 192, 2775, 2776, 3029, 2831, 3011]
df_selected = df.loc[specific_rows]
df_selected.head()

### Cleaning the startup_name column

In [ ]:
#Inspecting column
pd.set_option("display.max_rows", None)

value_counts = df['startup_name'].value_counts().sort_index()
print(value_counts)

In [ ]:
##Extracting the domain name from URL
#Function to extract domain name if the entry is a URL and capitalize the first letter
# Define the function for cleaning startup names
def clean_startup_name(value):
    # Remove non-ASCII characters
    value = value.encode('ascii', 'ignore').decode('utf-8')
    # Standardize specific patterns like "Byju's"
    value = re.sub(r"\"?Byju\\?'?S\"?", "Byjus", value, flags=re.IGNORECASE)
    
    # Check if the value is a URL
    if value.startswith('http'):
        domain_name = urlparse(value).netloc.split('.')[1].capitalize()  # Extract and capitalize domain
        return domain_name
    
    # Remove specific patterns for non-URL values
    cleaned_value = re.sub(r'\.com|\.co|\.in|\.ai|\.IO', '', value, flags=re.IGNORECASE)
    
    # Convert to title case
    return cleaned_value.title()

# Apply the combined cleaning function to the 'startup_name' column
df['startup_name'] = df['startup_name'].apply(clean_startup_name)
df.head()

# Select specific rows
specific_rows = [2269, 2873, 1677, 1411]
df_selected = df.loc[specific_rows]
df_selected.head()

In [ ]:
startup_mapping = {
    '1Mg (Healthkartplus)': '1Mg', 'Aahaa': 'Aahaa Stores', 'Absentia': 'Absentia Vr', 'Active': 'Active Ai',
    '#Fame': 'Fame', 'Arcatron': 'Arcatron Mobility', 'Ampere': 'Ampere Vehicles', 'Atomberg': 'Atomberg Technology',
    'Availfinance': 'Avail Finance', 'Ayefinance': 'Aye Finance', 'Appdaily': 'Appsdaily', 'Bank Bazaar': 'Bankbazaar',
    'Bhive': 'Bhive Workspace', 'Bhiveworkspace': 'Bhive Workspace', 'BigBasket': 'Bigbasket',
    'Black White Orange': 'Black White Orange Brands', 'Buildkar': 'Buildzar', 'BYJUs': 'Byjus', 'BYJUS': 'Byjus',
    'CCavenue': 'CCAvenue', 'Capillary': 'Capillary Tech', 'Caravan Craft Retail': 'Caravan Craft',
    'Chaipoint': 'Chai Point', 'Cloudcherry': 'Cloudcherry Analytics', 'Confirmttkt': 'Confirmtkt',
    'Craftstvilla': 'Craftsvilla', 'Crown-It': 'Crownit', 'Cult': 'Cult Fitness', 'Cure Fit': 'Curefit',
    'Daily Hunt': 'Dailyhunt', 'Daily Rounds': 'Dailyrounds', 'Deal 4Loans': 'Deal4Loans', 'Dhruva': 'Dhruva Space',
    'Doctorinsta': 'Doctor Insta', 'Early Salary': 'Earlysalary', 'Entropika': 'Entropik', 'Fab Hotels': 'Fabhotels',
    'Fyle Technologies': 'Fyle', 'Gibbs': 'Gibss', 'Glamstudios': 'Glam Studios', 'Hansel Io': 'Hansel',
    'Happilyunmarried': 'Happily Unmarried', 'Healthcare': 'Healthcare At Home', 'High Radius': 'Highradius',
    'Hike Messenger': 'Hike', 'Hwell24': 'Hwell24 Plus', 'I2I Funding': 'I2Ifunding', 'Impact Guru': 'Impactguru',
    'Incred': 'Incred Finance', 'Ink Monk': 'Inkmonk', 'Intelligencenode': 'Intelligence Node',
    'Jollyfoodfellow': 'Jolly Food Fellow', 'Legal Raasta': 'Legalraasta', 'Lenden Club': 'Lendenclub',
    'Lending Kart': 'Lendingkart', 'Little Black Book Delhi': 'Little Black Book', 'Loan Tap': 'Loantap',
    'Log 9 Materials': 'Log9 Materials', 'Logicroots': 'Logic Roots', 'Milk Basket': 'Milkbasket', 
    'Nearbuy (Previously Groupon India)': 'Nearbuy', 'Neogrowth Credit': 'Neogrowth', 'Olacabs': 'Ola Cabs',
    'Oyorooms': 'Oyo Rooms', 'Oyo': 'Oyo Rooms', 'Paytm Marketplace': 'Paytm', 'Pinelabs': 'Pine Labs', 
    'Pipabella': 'Pipa Bella', 'Policy Bazaar': 'Policybazaar', 'Pressplay Tv': 'Pressplay', 'Qyk App': 'Qyk', 
    'Rapido Bike Taxi': 'Rapido', 'Rawpressery':'Raw Pressery', 'Renew Buy': 'Renewbuy', 'Rentmojo': 'Rentomojo',
    'Rollmafia': 'Roll Mafia', 'Satvakart': 'Satvacart', 'Shabdnagari': 'Shabdanagari', 'Shubhloans': 'Shubh Loans',
    'Silvan': 'Silvan Innovation Labs', 'Samunnati Financial Intermediation & Services Pvt. Ltd': 'Samunnati',
    'Spares Hub': 'Spareshub', 'Sport Flashes': 'Sports Flashes', 'Stalk Buy Love': 'Stalkbuylove',
    'Staydobe': 'Stayabode', 'Tails Life': 'Tailslife', 'Tempgo': 'Tempogo', 'The Mons Co.': 'The Moms Co',
    'Toko Innovations': 'Toko Innovation Studios', 'Tone Tag': 'Tonetag', 'Ue Life Sciences': 'Ue Lifesciences',
    'Unaacademy': 'Unacademy', 'Urbanclap Technologies Pvt. Ltd': 'Urbanclap', 'Vahdam Tea': 'Vahdam Teas',
    'Veritas Finance': 'Veritas Finance Ltd.', 'Vogo': 'Vogo Automotive', 'Vogo Automotive Pvt. Ltd.': 'Vogo Automotive',
    'Zippserve': 'Zippserv', 'Zolo': 'Zolostays'   
}
df['startup_name'] = df['startup_name'].replace(startup_mapping)

In [ ]:
df.head()

In [ ]:
value_counts = df['startup_name'].value_counts().sort_index()
print(value_counts)

### Cleaning the city_location column

In [ ]:
value_counts = df['city_location'].value_counts().sort_index()
print(value_counts)

In [ ]:
city_mapping = {
    "Ahemadabad": "Ahmedabad",
    "Ahemdabad": "Ahmedabad",
    "Andheri": "Mumbai",
    "Bangalore": "Bengaluru",
    "Bangalore / Palo Alto": "Bengaluru",
    "Bangalore / SFO": "Bengaluru",
    "Bangalore / San Mateo": "Bengaluru",
    "Bangalore / USA": "Bengaluru",
    "Bangalore/ Bangkok": "Bengaluru",
    "Belgaum": "Bengaluru",
    "Bengaluru and Gurugram": "Bengaluru",
    "Bhubneswar":"Bhubaneswar",
    "Chembur": "Mumbai",
    "Chennai/ Singapore": "Chennai",
    "Dallas / Hyderabad": "Hyderabad",
    "Delhi & Cambridge": "Delhi",
    "Goa/Hyderabad": "Goa",
    "Gurgaon": "Gurugram",
    "Gurgaon / SFO": "Gurugram",
    "Hyderabad/USA": "Hyderabad",
    "India / US": "India",
    "India/Singapore": "India",
    "India/US": "India",
    "Karnataka": "Bengaluru",
    "Kochi": "Kerala",
    "Kolkatta": "Kolkata",
    "Kormangala": "Bengaluru",
    "Kozhikode": "Kerala",
    "Menlo Park": "California",
    "Mumbai/Bengaluru": "Mumbai",
    "Mumbai / Global": "Mumbai",
    "Mumbai / NY": "Mumbai",
    "Mumbai / UK": "Mumbai",
    "New Delhi / California": "New Delhi",
    "New Delhi / US": "New Delhi",
    "New Delhi/ Houston": "New Delhi",
    "Nw Delhi": "New Delhi",
    "New York": "New York",
    "New York, Bengaluru": "New York",
    "New York/ India": "New York",
    "Noida / Singapore": "Noida",
    "Palo Alto": "California",
    "Panaji": "Goa",
    "Pune / Dubai": "Pune",
    "Pune / Singapore": "Pune",
    "Pune / US": "Pune",
    "Pune/Seattle": "Pune",
    "San Francisco": "California",
    "San Jose,": "California",
    "Santa Monica": "California",
    "SFO / Bangalore": "Bengaluru", 
    "Seattle / Bangalore": "Bengaluru",
    "Taramani": "Chennai",
    "Trivandrum": "Kerala",
    "US": "USA",
    "US/India": "USA",
    "USA/India": "USA"
}
df['city_location'] = df['city_location'].replace(city_mapping)

In [ ]:
value_counts = df['city_location'].value_counts().sort_index()
print(value_counts)

### Cleaning the investment_type column

In [ ]:
#Inspecting column
pd.set_option("display.max_rows", None)
value_counts = df['investment_type'].value_counts().sort_index()
#print(value_counts)

In [ ]:
funding_mapping = {
    'Angel': 'Angel Funding',
    'Angel / Seed Funding': 'Angel Seed Funding',
    'Angel Round': 'Angel Funding',
    'Bridge Round': 'Bridge Funding',
    'Corporate Round': 'Corporate Funding',
    'Crowd Funding': 'Crowdfunding',
    'Crowd funding': 'Crowdfunding',
    'Debt': 'Debt Funding',
    'Debt-Funding': 'Debt Funding',
    'Debt and Preference capital': 'Debt Funding',
    'Equity': 'Equity Funding',
    'Equity Based Funding': 'Equity Funding',
    'Funding Round': 'Other Funding',
    'Inhouse Funding': 'Other Funding',
    'Maiden Round': 'Other Funding',
    'Mezzanine': 'Other Funding',
    'Pre Series A': 'Pre-Series A',
    'pre-series A': 'Pre-Series A',
    'pre-Series A': 'Pre-Series A',
    'Pre-series A': 'Pre-Series A',
    'Private': 'Private Equity',
    'Private Equity Round': 'Private Equity',
    'Private Funding': 'Private Equity',
    'PrivateEquity': 'Private Equity',
    'Seed': 'Seed Funding',
    'Seed / Angel Funding': 'Angel Seed Funding',
    'Seed / Angle Funding': 'Angel Seed Funding',
    'Seed Funding Round': 'Seed Funding',
    'Seed Round': 'Seed Funding',
    'Seed funding': 'Seed Funding',
    'Seed/ Angel Funding': 'Angel Seed Funding',
    'Seed/Angel Funding': 'Angel Seed Funding',
    'SeedFunding': 'Seed Funding',
    'Series B (Extension)':'Series B',
    'Single Venture': 'Venture Funding',
    'Structured Debt': 'Debt Funding',
    'Term Loan': 'Debt Funding',
    'Venture': 'Venture Funding',
    'Venture - Series Unknown': 'Venture Funding',
    'Venture Round': 'Venture Funding'
}

df['investment_type'] = df['investment_type'].replace(funding_mapping)

In [ ]:
value_counts = df['investment_type'].value_counts().sort_index()
print(value_counts)

### Cleaning the amount column

In [ ]:
def clean_amount(amount):
    amount = str(amount).strip().lower() #Converting to string and stripping spaces
    if amount in ['n/a', 'undisclosed', 'unknown']: 
        return 0.0

    #Regex to remove unwanted characters (except for digits, decimal point, and minus sign)
    amount = re.sub(r'[^0-9.-]', '', amount)
    try:
        cleaned_amount = float(amount)
    except ValueError:
        cleaned_amount = 0.0
    return cleaned_amount

df['amount_in_usd'] = df['amount_in_usd'].apply(clean_amount)

#Formatting the output to avoid scientific notation and keeping it as float
pd.set_option('display.float_format', '{:,.2f}'.format)
df.head()

In [ ]:
value_counts = df['amount_in_usd'].value_counts().sort_index()
print(value_counts)

In [ ]:
#df.sample(100)

In [ ]:
df.to_csv('C:/Users/Insight/Desktop/cleaned_data.csv', index=False) 